In [1]:
import torch
import matplotlib.pyplot as plt
import ESN
import time
import numpy as np
import json
import util

In [2]:
torch.set_default_device("cuda:0")
torch.set_default_dtype(torch.double)

In [3]:
fn = input("set file name : ")

set file name :  10N_100K


In [4]:
# Parameters
Two,Ttrain = 2000,100000
N = 10
rho=0.6
C = 0

dim = 1
sigma = 1.0
N_d = int(N * dim)
#N_d = 
shftreg = False
idwin = False
actf = "identity"
actf = "tanh"

rseed=0

setting = {"input dim":dim,"Two":Two, "Ttrain":Ttrain,"sigma":sigma,"rseed" : rseed, 
            "Nodes":N_d,"uC":C,"rho":rho, "actf":actf,"identical Win":idwin,"input dist":"uniform"}
# store experiment setting 
with open('./experiments/settings/'+fn+'_s.txt', 'x') as fp:
    data = json.dump(setting,fp)

print("settings stored:",setting)

settings stored: {'input dim': 1, 'Two': 2000, 'Ttrain': 100000, 'sigma': 1.0, 'rseed': 0, 'Nodes': 10, 'uC': 0, 'rho': 0.6, 'actf': 'tanh', 'identical Win': False, 'input dist': 'uniform'}


In [5]:
## create input
torch.manual_seed(0)
u_asym = torch.rand(dim, Two + Ttrain)
#u_sym = u_asym
u_sym = 2 * u_asym - 1

torch.save(u_sym,f"./experiments/inputs/{fn}_i.pt")



In [6]:
## construct ESN model
## run and washout 
esn = ESN.ESN_mult(N_d, rho=rho,uC=C, dim=dim,idWin = idwin,sigma =sigma,rseed=rseed)
st = time.time()
Xwo = esn.run_washout(u_sym, Two, actf=actf)
print("runtime :",time.time()-st)

torch.save(Xwo,f"./experiments/datamatrices/{fn}_d.pt")



runtime : 14.382144689559937


In [ ]:
""" calculate MEMORY CAPACITY """

with open('./experiments/settings/'+fn+'_s.txt', 'r') as fp:
    setting = json.load(fp)
u = torch.load('./experiments/inputs/'+fn+'_i.pt')
Xwo = torch.load('./experiments/datamatrices/'+fn+'_d.pt')

N = setting["Nodes"]
dim = setting["input dim"]
if setting["identical Win"]:
    maxtau = int(N*1.5) 
else : maxtau =  int(N/dim *2) 

""" tweak maxtau here """
##
maxtau=int(N*4)
##

#mfs = ESN.MC_cSVD_old(u_sym, Xwo, maxtau)
#mfs = ESN.MC_cSVD(u_sym, Xwo, maxtau)
mfs_raw, mfs_lin, mfs_rev, sur_value = ESN.MC_cSVD(u_sym, Xwo, maxtau,ret_all=True)
mfs = mfs_rev

print("mc calc time :",time.time()-st)
print("total MC :",float(torch.sum(mfs)))
torch.save(mfs,f"./experiments/mfs/{fn}_mf.pt")
torch.save(mfs_raw,f"./experiments/mfs/{fn}_raw.pt")
torch.save(sur_value,f"./experiments/mfs/{fn}_sur.pt")

print(float(sur_value))


In [7]:
## calculate ipc

## create target functions
with open('./experiments/settings/'+fn+'_s.txt', 'r') as fp:
    setting = json.load(fp)
u_sym = torch.load('./experiments/inputs/'+fn+'_i.pt')
Xwo = torch.load('./experiments/datamatrices/'+fn+'_d.pt')

Two = setting["Two"]

## set max delay, degree
maxdd=[[1,20],
       [2,10],
       [3,10],
       [4,7],
       [5,7],
       [6,3],
       [7,3]]


st = time.time()
target_info = ESN.make_targets(u_sym,maxdd,Two=Two)
print(r"target creation time:%.3f s"%(time.time()-st))
torch.save(target_info,f"./experiments/target_info/{fn}_ti.pt")



/tmp/ipykernel_194041/2217200234.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  u_sym = torch.load('./experiments/inputs/'+fn+'_i.pt')
/tmp/ipykernel_194041/2217200234.

basis table creation:0.261 s
1 degree:20 target functions
2 degree:55 target functions
3 degree:220 target functions
4 degree:210 target functions
5 degree:462 target functions
6 degree:28 target functions
7 degree:36 target functions
Total of 1031 bases
 7/7 degree bases now calculating ...target creation complete, total of 1031 bases
target creation time:6.070 s


In [8]:
## calculate capacities
#
st = time.time()
#ti = torch.load('./experiments/target_info/'+fn+'_ti.pt')
print(r"load time :%.3f s"%(time.time()-st))
ti=target_info
st = time.time()
raw,thr,thr_scl,rev,sur= ESN.calc_capacity(Xwo,ti.tar_f,ret_all=True,thr_scale=1.2)
print(r"ipc calc time :%.3f s"%(time.time()-st))

cap = thr

ipc=ESN.IPC(cap,ti.delay,ti.degree,ti.in_dim,ti.maxddset)


torch.save(ipc,f"./experiments/ipcs/{fn}_ipc.pt")
torch.save(raw,f"./experiments/ipcs/{fn}_raw.pt")
torch.save(sur,f"./experiments/ipcs/{fn}_sur.pt")

util.print_ipc(ipc)

load time :0.000 s
ipc calc time :0.243 s


In [9]:
## print ipc
degree_num=len(ipc.maxddset)
cap_deg=np.array(())
for deg in range(degree_num):
    tempcap = torch.sum(ipc.get_val_by_degree(deg+1)).cpu().numpy()
    cap_deg = np.append(cap_deg,tempcap)
    print(f"{deg+1} deg ipc:",tempcap)
print("total ipc:",float(np.sum(cap_deg)))


1 deg ipc: 5.580646651153414
2 deg ipc: 0.0004904261897487859
3 deg ipc: 4.350434935313661
4 deg ipc: 0.0
5 deg ipc: 0.059215086439763295
6 deg ipc: 0.0
7 deg ipc: 0.0
total ipc: 9.990787099096586
